# Lab 3: IR

## Install, import modules and download dataset

In [1]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

     |████████████████████████████████| 471kB 2.8MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.4-cp36-cp36m-linux_x86_64.whl size=275761 sha256=7eee7e28e98d296174f37054880a88351a40f07d8f89c218d9d100c42eb91bc1
  Stored in directory: /root/.cache/pip/wheels/58/30/73/8858a1b6e5e2674e2ea85c9904949c06addcf6fd34d59b5ea6
Successfully built pytrec-eval
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=7db6bcb696cf40a553414b7d3ce9a004e17976f91c2d5099a32c1b8ea1ed24e9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
import os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval#didnt use in assignment?
import wget#

In [0]:
filename = wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/lab-data.zip", "lab-data.zip")

In [4]:
!unzip lab-data.zip

Archive:  lab-data.zip
   creating: lab-data/
  inflating: lab-data/air.topics     
   creating: lab-data/documents/
  inflating: lab-data/documents/email05  
  inflating: lab-data/documents/email02  
  inflating: lab-data/documents/email03  
  inflating: lab-data/documents/email04  
  inflating: lab-data/documents/email10  
  inflating: lab-data/documents/email08  
  inflating: lab-data/documents/email01  
  inflating: lab-data/documents/email06  
  inflating: lab-data/documents/email07  
  inflating: lab-data/documents/email09  
  inflating: lab-data/documents/email14  
  inflating: lab-data/air.qrels      


In [0]:
DATA_DIR = "lab-data"
DOCUMENTS_DIR = os.path.join(DATA_DIR, "documents")
TOPIC_FILE = os.path.join(DATA_DIR, "air.topics")
QRELS_FILE = os.path.join(DATA_DIR, "air.qrels")

## Part 1: Basic Indexing

### Creating the index

In [0]:
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()#make temp dict

    # create and return the index
    return index.create_in(indexDir, schema)

In [0]:
# first, define a Schema for the index
mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))#only edit analyzer=''''''step 1 done

# now, create the index at the path INDEX_DIR based on the new schema
myIndex = createIndex(mySchema)

### Indexing the documents

In [0]:
def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):#go through all files and add them
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1 % 1000 == 0):
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

In [0]:
# Build a list of files to index
filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]

In [10]:
# Check the list
filesToIndex[:11]

['lab-data/documents/email01',
 'lab-data/documents/email06',
 'lab-data/documents/email10',
 'lab-data/documents/email07',
 'lab-data/documents/email09',
 'lab-data/documents/email05',
 'lab-data/documents/email14',
 'lab-data/documents/email04',
 'lab-data/documents/email08',
 'lab-data/documents/email02',
 'lab-data/documents/email03']

In [11]:
# count files to index
print("number of files:", len(filesToIndex))

number of files: 11


In [12]:
addFilesToIndex(myIndex, filesToIndex)#step 2 done, assignment may take 3mins to finish

done indexing.


### Querying

In [0]:
# define a query parser for the field "file_content" in the index
myQueryParser = QueryParser("file_content", schema=myIndex.schema)
mySearcher = myIndex.searcher()

In [14]:
# run a sample query for the phrase "item"
sampleQuery = myQueryParser.parse("item")#item to duck,,,,try
sampleQueryResults = mySearcher.search(sampleQuery, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults):
    score = sampleQueryResults.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)#step 3 done

email01 0 2.6746417187049216


### Evaluation using TREC_EVAL
In order to evaluate our results we will use a topic file - a list of topics we use to evaluate our IR system

In [15]:
# print the topic file
with open(TOPIC_FILE, "r") as f:
    print(f.read())

01 ducks
02 ig nobel prizes
03 mathematics
04 flowing hair
05 music
06 AIR TV



We will compare our evaluate our results with a set of judged results(qrels file) using TREC_EVAL 

In [16]:
# print the first 10 lines in the qrels file
with open(QRELS_FILE, "r") as f:
    qrels10 = f.readlines()[:10]
    print("".join(qrels10))

01 0 email01 0
01 0 email02 0
01 0 email03 0
01 0 email04 1
01 0 email05 1
01 0 email06 1
01 0 email07 0
01 0 email08 0
01 0 email09 0
01 0 email10 0



The follwing function takes a topic file, a qrels file, a query parser and a searcher and use TREC_EVAL to compare our results with the provided qrels file (see assignment PDF for more details)

In [0]:
def pyTrecEval(topicFile, qrelsFile, queryParser, searcher):#question 4, if want higher marks ,go through
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure == "runid":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

In [18]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser, mySearcher) 

num_q                    01      1.0000
num_ret                  01      1.0000
num_rel                  01      3.0000
num_rel_ret              01      1.0000
map                      01      0.3333
gm_map                   01      -1.0986
Rprec                    01      0.3333
bpref                    01      0.3333
recip_rank               01      1.0000
iprec_at_recall_0.00     01      1.0000
iprec_at_recall_0.10     01      1.0000
iprec_at_recall_0.20     01      1.0000
iprec_at_recall_0.30     01      1.0000
iprec_at_recall_0.40     01      0.0000
iprec_at_recall_0.50     01      0.0000
iprec_at_recall_0.60     01      0.0000
iprec_at_recall_0.70     01      0.0000
iprec_at_recall_0.80     01      0.0000
iprec_at_recall_0.90     01      0.0000
iprec_at_recall_1.00     01      0.0000
P_5                      01      0.2000
P_10                     01      0.1000
P_15                     01      0.0667
P_20                     01      0.0500
P_30                     01      0.0333

## Part 2: Evaluating different configurations

### Inspecting our index

In [19]:
# Is it empty?
print("Index is empty?", myIndex.is_empty())

# How many files indexed?
print("Number of indexed files:", myIndex.doc_count())

Index is empty? False
Number of indexed files: 11


In [0]:
# define a reader object on the index
myReader = myIndex.reader()

In [21]:
# print first 5 indexed documents
[(docnum, doc_dict) for (docnum, doc_dict) in myReader.iter_docs()][0:5]

[(0, {'file_path': 'lab-data/documents/email01'}),
 (1, {'file_path': 'lab-data/documents/email06'}),
 (2, {'file_path': 'lab-data/documents/email10'}),
 (3, {'file_path': 'lab-data/documents/email07'}),
 (4, {'file_path': 'lab-data/documents/email09'})]

In [22]:
# list indexed terms for field "file_content"
[term for term in myReader.field_terms("file_content")][1000:1025]

['Care',
 'Carlos',
 'Carmen',
 'Carnivalesque',
 'Carolina',
 'Case',
 'Cat',
 'Catalysis',
 'Catalyst',
 'Catchers',
 'Cater',
 'Caused',
 'Caveat',
 'CbZF1d0021swQuc57kfqHt',
 'Cechetto',
 'Ceder',
 'Celebratory',
 'Center',
 'Cereal',
 'Ceremony',
 'Cerrahi',
 'Certolizumab',
 'Cervical',
 'Chair',
 'Chalfie']

In [23]:
#how many terms do we have?
print(myReader.field_length("file_content"))

29729


In [24]:
# how many documents have the phares "bit", blob"
#   in the field "file_content"?
print("# docs with 'bit'", myReader.doc_frequency("file_content", "bit"))
print("# docs with 'are'", myReader.doc_frequency("file_content", "are"))
print("# docs with 'get'", myReader.doc_frequency("file_content", "get"))

# docs with 'bit' 1
# docs with 'are' 11
# docs with 'get' 6


### Text Analyzers

In [25]:
# we start with basic tokenizer
tokenizer = RegexTokenizer()
[token.text for token in tokenizer("We are going to do Text Analysis with whoosh.analysis")]

['We',
 'are',
 'going',
 'to',
 'do',
 'Text',
 'Analysis',
 'with',
 'whoosh.analysis']

In [26]:
# we might want use stemming:
stmAnalyzer = RegexTokenizer() | StemFilter()
[token.text for token in stmAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

['We', 'ar', 'go', 'to', 'do', 'Text', 'Analysi', 'with', 'whoosh.analysi']

In [27]:
# We probably want to lower-case it
# so we add LowercaseFilter
stmLwrAnalyzer = RegexTokenizer() | LowercaseFilter() | StemFilter()
[token.text for token in stmLwrAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

['we', 'ar', 'go', 'to', 'do', 'text', 'analysi', 'with', 'whoosh.analysi']

In [28]:
# we probably want to ignore words like "we", "are", "with" when we index files
# so we add StopFilter to filter stop words
stmLwrStpAnalyzer = RegexTokenizer() | LowercaseFilter() | StopFilter() | StemFilter()
[token.text for token in stmLwrStpAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

['go', 'do', 'text', 'analysi', 'whoosh.analysi']

In [29]:
# we also probably want to break phrases like "whoosh.analysis" into "whoosh" and "analysis"
# so we add IntraWordFilter
stmLwrStpIntraAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | StemFilter()
[token.text for token in stmLwrStpIntraAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

['go', 'do', 'text', 'analysi', 'whoosh', 'analysi']

### Evaluating the new analyzers

In [0]:
# define a Schema with the new analyzer
mySchema2 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = stmLwrStpIntraAnalyzer))

# create the index based on the new schema
myIndex2 = createIndex(mySchema2)

In [31]:
addFilesToIndex(myIndex2, filesToIndex)

done indexing.


In [0]:
# define a query parser for the field "file_content" in the index
myQueryParser2 = QueryParser("file_content", schema=myIndex2.schema)
mySearcher2 = myIndex2.searcher()

In [33]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, myQueryParser2, mySearcher2) 

num_q                    01      1.0000
num_ret                  01      3.0000
num_rel                  01      3.0000
num_rel_ret              01      3.0000
map                      01      1.0000
gm_map                   01      0.0000
Rprec                    01      1.0000
bpref                    01      1.0000
recip_rank               01      1.0000
iprec_at_recall_0.00     01      1.0000
iprec_at_recall_0.10     01      1.0000
iprec_at_recall_0.20     01      1.0000
iprec_at_recall_0.30     01      1.0000
iprec_at_recall_0.40     01      1.0000
iprec_at_recall_0.50     01      1.0000
iprec_at_recall_0.60     01      1.0000
iprec_at_recall_0.70     01      1.0000
iprec_at_recall_0.80     01      1.0000
iprec_at_recall_0.90     01      1.0000
iprec_at_recall_1.00     01      1.0000
P_5                      01      0.6000
P_10                     01      0.3000
P_15                     01      0.2000
P_20                     01      0.1500
P_30                     01      0.1000


In [34]:
# let count the same words again
myReader2 = myIndex2.reader()
print("# docs with 'bit'", myReader2.doc_frequency("file_content", "bit"))
print("# docs with 'are'", myReader2.doc_frequency("file_content", "are"))
print("# docs with 'get'", myReader2.doc_frequency("file_content", "get"))

# docs with 'bit' 11
# docs with 'are' 0
# docs with 'get' 7


**Can you explain the differences?**

### Using NLTK's stemmers and lemmatizers

In [0]:
import nltk
from nltk.stem import *

In [36]:
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
# we'll compare two stemmers and a lemmatizer
lrStem = LancasterStemmer()
sbStem = SnowballStemmer("english")
wnLemm = WordNetLemmatizer()

In [0]:
# define a list of words to compare the stemmers on
listWords = ["going", "saying", "minimize", "maximum", 
             "meeting", "files", "tries", "is", "are", "beautiful",
             "summarize", "better", "dogs", "phenomena"]

In [39]:
for word in listWords:
    print("%15s %15s %15s %15s" % (lrStem.stem(word),
                                   sbStem.stem(word),
                                   wnLemm.lemmatize(word),
                                   wnLemm.lemmatize(word, 'v')))

          going              go           going              go
            say             say          saying             say
          minim           minim        minimize        minimize
          maxim         maximum         maximum         maximum
           meet            meet         meeting            meet
            fil            file            file            file
            tri             tri             try             try
             is              is              is              be
             ar             are             are              be
         beauty          beauti       beautiful       beautiful
           summ          summar       summarize       summarize
            bet          better          better          better
            dog             dog             dog             dog
       phenomen       phenomena      phenomenon       phenomena


### How to use NLTK stemmers / lemmatizers in Whoosh

In [0]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [41]:
# Example1: Whoosh filter for NLTK's LancasterStemmer
myFilter1 = RegexTokenizer() | CustomFilter(LancasterStemmer().stem)
[token.text for token in myFilter1("We are going to do Text Analysis with whoosh.analysis")]

['we', 'ar', 'going', 'to', 'do', 'text', 'analys', 'with', 'whoosh.analysis']

In [42]:
# Example2: Whoosh filter for NLTK's WordNetLemmatizer
myFilter2 = RegexTokenizer() | CustomFilter(WordNetLemmatizer().lemmatize)
[token.text for token in myFilter2("We are going to do Text Analysis with whoosh.analysis")]

['We',
 'are',
 'going',
 'to',
 'do',
 'Text',
 'Analysis',
 'with',
 'whoosh.analysis']

In [43]:
# Example3: Whoosh filter for NLTK's WordNetLemmatizer for verbs
myFilter3 = RegexTokenizer() | CustomFilter(WordNetLemmatizer().lemmatize, 'v')
[token.text for token in myFilter3("We are going to do Text Analysis with whoosh.analysis")]

['We', 'be', 'go', 'to', 'do', 'Text', 'Analysis', 'with', 'whoosh.analysis']

You can now use myFilter1/2/3 as part of your Schema

------------
You can find details of other NLTK Stemmers and Lemmatizers here:

http://www.nltk.org/api/nltk.stem.html